In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import random
from glob import glob
from tqdm import tqdm
from scipy.io import loadmat

import torch
from torch import nn
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from sklearn.metrics import classification_report

In [2]:
data_dir = "/media/mountHDD3/data_storage/biomedical_data/ecg_data/SPH"
print(os.listdir(data_dir))

['metadata.csv', 'data_df.csv', 'data_df_no1.csv', 'records']


In [3]:
main_df = pd.read_csv(data_dir + "/data_df.csv")
main_df.shape

(20792, 4)

In [4]:
single_fns = main_df["File name"].values.tolist()
single_mat_paths = [data_dir + f"/records/{x}.h5" for x in single_fns]

In [5]:
ratio = [0.8, 0.1]

train_index = int(len(single_mat_paths)*ratio[0])
valid_index = int(len(single_mat_paths)*(ratio[0]+ratio[1]))

train_mat_paths = single_mat_paths[:train_index]
valid_mat_paths = single_mat_paths[valid_index:]

train_label = main_df
valid_label = main_df

In [6]:
conv_block1 = nn.Sequential(
			nn.Conv1d(12, 64, 16, stride=1),
			nn.LeakyReLU(0.3),
			nn.BatchNorm1d(64),
		)

class Redu(nn.Module):
    def __init__(self, in_channel, out_channel, kernel_size, padding):
        super().__init__()
        
        self.in_channel = in_channel
        self.out_channel = out_channel
        self.kernel_size = kernel_size
        self.padding = padding
        
        self.conv1 = nn.Sequential(
            nn.MaxPool1d(4),
            nn.Conv1d(self.in_channel, self.out_channel, 1, 1)
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(self.in_channel, self.out_channel, self.kernel_size, 1),
            nn.LeakyReLU(),
            nn.BatchNorm1d(self.out_channel),
            nn.Dropout(0.2),
            nn.Conv1d(self.out_channel, self.out_channel, self.kernel_size, 4, self.padding)
        )
        self.conv3 = nn.Sequential(
            nn.LeakyReLU(),
            nn.BatchNorm1d(self.out_channel),
            nn.Dropout(0.2),
        )
    
    def forward(self, x1, x2):
        x1 = self.conv1(x1)
        x2 = self.conv2(x2)
        x_res = x1 + x2
        x3 = self.conv3(x_res)
        
        return x_res, x3

In [7]:
test = torch.randn(1, 12, 1400)
test = conv_block1(test)
model = Redu(64, 128, 3, 0)
a1, a2 = model(test, test)
print(type(a2))

<class 'torch.Tensor'>


In [8]:
class GlobalAvgPooling(nn.Module):
    def __init__(self):
        super(GlobalAvgPooling, self).__init__()

    def forward(self, x):
        return x.mean(dim=(2))

In [9]:
class LSAT(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.lstm = nn.LSTM(5, 5)
        
        self.conv1 = nn.Sequential(
            nn.Conv1d(960, 256, 1, 1),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.5),
        )
    
        self.conv2 = nn.Sequential(
            nn.Conv1d(256, 128, 1, 1),
            nn.BatchNorm1d(128),
            nn.Dropout(0.5),                 
        )
        
        self.attention = nn.MultiheadAttention(5, 1, batch_first=True)
        
        self.conv3 = nn.Sequential(
            nn.LeakyReLU(0.3),
            nn.BatchNorm1d(128),
            nn.Dropout(0.5),
        )
        
        self.avgpo = GlobalAvgPooling()
            
        self.conv4 = nn.Sequential(
            nn.Linear(128, 64),
            nn.LeakyReLU(0.3),
            nn.Linear(64, 32),
            nn.LeakyReLU(0.3),
            nn.Linear(32, 34),
            nn.Sigmoid(),
        )
    
    def forward(self, x):
        x1, _ = self.lstm(x)
        x1 = self.conv1(x1)
        
        x1, _ = self.lstm(x1)
        x2 = self.conv2(x1)
        
        x3, _ = self.attention(x2, x2, x2)
        x3 = self.conv3(x3)
        
        x3 = self.avgpo(x2)
        x4 = self.conv4(x3)
        
        return x4   
    

In [10]:
import h5py

class HeartData(Dataset):
    def __init__(self, data_paths):
        self.data_paths = data_paths
        random.shuffle(self.data_paths)

    def __getitem__(self, idx):
        data_path = self.data_paths[idx]
        a = h5py.File(data_path, 'r')
        data_h5 = a['ecg']
        data = np.array(data_h5)
        clip_data = data[:, -1400:]

        filename = data_path.split("/")[-1].split(".")[0]
        label = main_df[main_df["File name"] == filename]["New Label"].values.item()

        torch_data = torch.from_numpy(clip_data)

        return torch_data.float(), label

    def __len__(self):
        return len(self.data_paths)

In [11]:
train_ds = HeartData(train_mat_paths)
valid_ds = HeartData(valid_mat_paths)

In [12]:
train_dl = DataLoader(train_ds, batch_size = 64, shuffle = True, pin_memory = True, num_workers = 48)
valid_dl = DataLoader(valid_ds, batch_size = 64, shuffle = True, pin_memory = True, num_workers = 48)

In [13]:
class HeartDNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv_block1 = nn.Sequential(
			nn.Conv1d(12, 64, 16, stride=1),
			nn.LeakyReLU(0.3),
			nn.BatchNorm1d(64),
		)
        self.model_redu11 = Redu(64, 128, 3, 0)
        self.model_redu12 = Redu(128, 192, 3, 0)
        self.model_redu13 = Redu(192, 256, 3, 0)
        self.model_redu14 = Redu(256, 320, 3, 0)
        
        self.model_redu21 = Redu(64, 128, 5, 2)
        self.model_redu22 = Redu(128, 192, 5, 2)
        self.model_redu23 = Redu(192, 256, 5, 2)
        self.model_redu24 = Redu(256, 320, 5, 2)
        
        self.model_redu31 = Redu(64, 128, 7, 4)
        self.model_redu32 = Redu(128, 192, 7, 4)
        self.model_redu33 = Redu(192, 256, 7, 4)
        self.model_redu34 = Redu(256, 320, 7, 4)
        
        self.model_LSAT = LSAT()
    
    
    def forward(self, x):  
    
        data_conv = self.conv_block1(x)

        data_x11, data_x12 = self.model_redu11(data_conv, data_conv)
        data_x13, data_x14 = self.model_redu12(data_x11, data_x12)
        data_x15, data_x16 = self.model_redu13(data_x13, data_x14)
        data_x17, data_x18 = self.model_redu14(data_x15, data_x16)

        
        data_x21, data_x22 = self.model_redu21(data_conv, data_conv)
        data_x23, data_x24 = self.model_redu22(data_x21, data_x22)
        data_x25, data_x26 = self.model_redu23(data_x23, data_x24)
        data_x27, data_x28 = self.model_redu24(data_x25, data_x26)

        
        data_x31, data_x32 = self.model_redu31(data_conv, data_conv)
        data_x33, data_x34 = self.model_redu32(data_x31, data_x32)
        data_x35, data_x36 = self.model_redu33(data_x33, data_x34)
        data_x37, data_x38 = self.model_redu34(data_x35, data_x36)
        
        data = torch.cat((data_x18, data_x28, data_x38), dim = 1)
   
        out = self.model_LSAT(data)

        return out

In [14]:
# test = torch.randn(1, 12, 1400)
model = HeartDNN()

In [15]:
class FocalClassifierV0(nn.Module):
    def __init__(self, gamma=0.3): #Change gamma value here in order to acquire other results
        super().__init__()
        
        self.gamma = gamma
        self.act = nn.LogSoftmax(dim=1)

    
    def forward(self, pred, target):

        logits = self.act(pred)

        B, C = tuple(logits.size())

        entropy = torch.pow(1 - logits, self.gamma) * logits * F.one_hot(target, num_classes=C).float()

        return (-1 / B) * torch.sum(entropy)

focalloss_fn = FocalClassifierV0()

In [16]:
epoch = 150
lr = 0.001
best_acc = 0
best_ep = 0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu", index = 0)
model.to(device)
optimizer = Adam(model.parameters(), lr=lr, betas = (0.9, 0.999), weight_decay = 0.001)
loss_fn = nn.CrossEntropyLoss()

In [17]:
for e in range(epoch):
    model.train()
    print(f"Epoch: {e}")
    y_true_list = [] 
    pred_list = []
    batch_cnt = 0
    total_loss = 0
    correct = 0
    for batch, (train_sig, train_label) in tqdm(enumerate(train_dl)):
        batch_cnt = batch
        train_sig = train_sig.to(device)
        train_label = train_label.to(device)
        
        pred = model(train_sig)
        loss = focalloss_fn(pred, train_label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
                
        total_loss += loss.item()
        correct += (pred.argmax(1) == train_label).type(torch.float).sum().item()
    
    total_loss /= batch_cnt
    correct /= len(train_dl.dataset)
    
    print(f"train loss: {total_loss} - train acc: {100*correct}")
    
    batch_cnt = 0
    val_total_loss = 0
    val_correct = 0
    model.eval()
    with torch.no_grad():
        for batch, (valid_sig, valid_label) in tqdm(enumerate(valid_dl)):
            batch_cnt = batch
            valid_sig = valid_sig.to(device)
            valid_label = valid_label.to(device)
            
            pred = model(valid_sig)
            
            pred_pos = pred.argmax(1)
            y_true_list.append(valid_label)
            pred_list.append(pred_pos)
            
            loss = loss_fn(pred, valid_label)
            
            val_total_loss += loss.item()
            val_correct += (pred.argmax(1) == valid_label).type(torch.float).sum().item()
    
        val_total_loss /= batch_cnt
        val_correct /= len(valid_dl.dataset)
        if val_correct > best_acc:
            best_acc = val_correct
            best_ep = e
        
        print(f"valid loss: {val_total_loss} - valid acc: {100*val_correct}")
        
y_true = torch.cat(y_true_list).cpu().numpy()
pred = torch.cat(pred_list).cpu().numpy()

reports = classification_report(y_true, pred, output_dict=True) 

print(reports)
print(f"Best acuracy: {best_acc} at epoch {best_ep}")

Epoch: 0


0it [00:00, ?it/s]

1it [00:01,  1.41s/it]

5it [00:01,  4.21it/s]

9it [00:01,  8.03it/s]

...

27it [00:00, 30.83it/s]

31it [00:01, 31.17it/s]

33it [00:01, 22.60it/s]

valid loss: 2.8538406416773796 - valid acc: 67.21153846153847
{'0': {'precision': 0.6927029804727647, 'recall': 0.9810771470160117, 'f1-score': 0.8120481927710844, 'support': 1374.0}, '1': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 48.0}, '2': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 130.0}, '3': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 86.0}, '4': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 13.0}, '5': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 3.0}, '6': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 10.0}, '7': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 20.0}, '8': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 2.0}, '9': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 8.0}, '10': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 4.0}, '11': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 82.0}, '12


/media/mountHDD2/thao/git/.env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/mountHDD2/thao/git/.env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/mountHDD2/thao/git/.env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
